# DE1 — Lab 2: PostgreSQL → Star Schema ETL
> Author : Badr TAJINI - Data Engineering I - ESIEE 2025-2026
---


Execute all cells. Attach evidence and fill metrics.

## 0. Setup and schemas

In [44]:
from pyspark.sql import SparkSession, functions as F, types as T
spark = SparkSession.builder.appName("de1-lab2").getOrCreate()
base = "data/"
# Explicit schemas
customers_schema = T.StructType([
    T.StructField("customer_id", T.IntegerType(), False),
    T.StructField("name", T.StringType(), True),
    T.StructField("email", T.StringType(), True),
    T.StructField("created_at", T.TimestampType(), True),
])
brands_schema = T.StructType([
    T.StructField("brand_id", T.IntegerType(), False),
    T.StructField("brand_name", T.StringType(), True),
])
categories_schema = T.StructType([
    T.StructField("category_id", T.IntegerType(), False),
    T.StructField("category_name", T.StringType(), True),
])
products_schema = T.StructType([
    T.StructField("product_id", T.IntegerType(), False),
    T.StructField("product_name", T.StringType(), True),
    T.StructField("brand_id", T.IntegerType(), True),
    T.StructField("category_id", T.IntegerType(), True),
    T.StructField("price", T.DoubleType(), True),
])
orders_schema = T.StructType([
    T.StructField("order_id", T.IntegerType(), False),
    T.StructField("customer_id", T.IntegerType(), True),
    T.StructField("order_date", T.TimestampType(), True),
])
order_items_schema = T.StructType([
    T.StructField("order_item_id", T.IntegerType(), False),
    T.StructField("order_id", T.IntegerType(), True),
    T.StructField("product_id", T.IntegerType(), True),
    T.StructField("quantity", T.IntegerType(), True),
    T.StructField("unit_price", T.DoubleType(), True),
])


In [32]:
import os

# Affiche le chemin absolu du répertoire courant
print(os.getcwd())

/mnt/c/Users/Justine/Data_engineering/lab2


## 1. Ingest operational tables (from CSV exports)

In [45]:
customers = spark.read.schema(customers_schema).option("header","true").csv(base+"lab2_customers.csv")
brands = spark.read.schema(brands_schema).option("header","true").csv(base+"lab2_brands.csv")
categories = spark.read.schema(categories_schema).option("header","true").csv(base+"lab2_categories.csv")
products = spark.read.schema(products_schema).option("header","true").csv(base+"lab2_products.csv")
orders = spark.read.schema(orders_schema).option("header","true").csv(base+"lab2_orders.csv")
order_items = spark.read.schema(order_items_schema).option("header","true").csv(base+"lab2_order_items.csv")

for name, df in [("customers",customers),("brands",brands),("categories",categories),("products",products),("orders",orders),("order_items",order_items)]:
    print(name, df.count())


customers 24
brands 8
categories 9
products 60
orders 220
order_items 638


In [34]:
import requests
import pandas as pd
import os
from datetime import datetime

def export_spark_metrics_lab2(spark, run_id, task, note="", output_csv="lab2_metrics_log.csv"):
    """
    Exporte les métriques Spark (stages) pour Lab2 dans un CSV.
    - spark: SparkSession active
    - run_id: identifiant du run (ex: "r1")
    - task: nom de la tâche (ex: "ingest_plan")
    - note: commentaire ou variante testée
    - output_csv: nom du fichier de sortie
    """
    # Trigger computation pour s'assurer que Spark a fini le job
    try:
        spark.sparkContext.statusTracker().getJobIdsForGroup(None)
    except:
        pass
    
    ui_url = spark.sparkContext.uiWebUrl
    app_id = spark.sparkContext.applicationId
    
    if not ui_url:
        print("Pas d'UI Spark détectée (aucun job en cours ?)")
        return
    
    try:
        stages_url = f"{ui_url}/api/v1/applications/{app_id}/stages"
        stages = requests.get(stages_url).json()
    except Exception as e:
        print(f"Erreur API Spark: {e}")
        return
    
    data = []
    for s in stages:
        stage_id = s.get("stageId")
        job_ids = s.get("jobIds", [])
        
        data.append({
            "run_id": task,  # on met le nom de la task ici
            "task": task,
            "note": note,
            "files_read": s.get("inputRecords", 0),
            "input_size_bytes": s.get("inputBytes", 0),
            "shuffle_read_bytes": s.get("shuffleReadBytes", 0),
            "shuffle_write_bytes": s.get("shuffleWriteBytes", 0),
            "timestamp": datetime.now().isoformat()
        })
    
    df = pd.DataFrame(data)
    
    if df.empty:
        print(f"Aucune métrique trouvée pour {task} (job peut-être pas encore exécuté)")
        return
    
    # Écrire ou ajouter au CSV
    if os.path.exists(output_csv):
        df.to_csv(output_csv, mode="a", header=False, index=False)
    else:
        df.to_csv(output_csv, index=False)
    
    print(f"{len(df)} lignes exportées vers {output_csv} pour la tâche {task}")


### Evidence: ingestion plan

In [46]:
ingest = orders.join(order_items, "order_id").select("order_id").distinct()
ingest.explain("formatted")
export_spark_metrics_lab2(spark, run_id="r1", task="ingest_plan", note="ingest operational tables")

from datetime import datetime as _dt
import pathlib
pathlib.Path("proof").mkdir(exist_ok=True)
with open("proof/plan_ingest.txt","w") as f:
    f.write(str(_dt.now())+"\n")
    f.write(ingest._jdf.queryExecution().executedPlan().toString())
print("Saved proof/plan_ingest.txt")

== Physical Plan ==
AdaptiveSparkPlan (11)
+- HashAggregate (10)
   +- Exchange (9)
      +- HashAggregate (8)
         +- Project (7)
            +- BroadcastHashJoin Inner BuildLeft (6)
               :- BroadcastExchange (3)
               :  +- Filter (2)
               :     +- Scan csv  (1)
               +- Filter (5)
                  +- Scan csv  (4)


(1) Scan csv 
Output [1]: [order_id#403]
Batched: false
Location: InMemoryFileIndex [file:/mnt/c/Users/Justine/Data_engineering/lab2/data/lab2_orders.csv]
PushedFilters: [IsNotNull(order_id)]
ReadSchema: struct<order_id:int>

(2) Filter
Input [1]: [order_id#403]
Condition : isnotnull(order_id#403)

(3) BroadcastExchange
Input [1]: [order_id#403]
Arguments: HashedRelationBroadcastMode(List(cast(input[0, int, false] as bigint)),false), [plan_id=3615]

(4) Scan csv 
Output [1]: [order_id#407]
Batched: false
Location: InMemoryFileIndex [file:/mnt/c/Users/Justine/Data_engineering/lab2/data/lab2_order_items.csv]
PushedFilters: [IsNotN

## 2. Surrogate key function

In [48]:
def sk(cols):
    # stable 64-bit positive surrogate key from natural keys
    return F.abs(F.xxhash64(*[F.col(c) for c in cols]))


## 3. Build dimensions

In [49]:
dim_customer = customers.select(
    sk(["customer_id"]).alias("customer_sk"),
    "customer_id","name","email","created_at"
)

dim_brand = brands.select(
    sk(["brand_id"]).alias("brand_sk"),
    "brand_id","brand_name"
)

dim_category = categories.select(
    sk(["category_id"]).alias("category_sk"),
    "category_id","category_name"
)

dim_product = products.select(
    sk(["product_id"]).alias("product_sk"),
    "product_id","product_name",
    sk(["brand_id"]).alias("brand_sk"),
    sk(["category_id"]).alias("category_sk"),
    "price"
)


## 4. Build date dimension

In [50]:
from pyspark.sql import Window as W
dates = orders.select(F.to_date("order_date").alias("date")).distinct()
dim_date = dates.select(
    sk(["date"]).alias("date_sk"),
    F.col("date"),
    F.year("date").alias("year"),
    F.month("date").alias("month"),
    F.dayofmonth("date").alias("day"),
    F.date_format("date","E").alias("dow")
)


## 5. Build fact_sales with broadcast joins where appropriate

In [51]:
oi = order_items.alias("oi")
p = products.alias("p")
o = orders.alias("o")
c = customers.alias("c")

# Join with small dimensions using DF copies to compute SKs, then broadcast dims by size heuristic
df_fact = (oi
    .join(p, F.col("oi.product_id")==F.col("p.product_id"))
    .drop(F.col("p.product_id"))
    .join(o, "order_id")
    .join(c, "customer_id")
    .withColumn("date", F.to_date("order_date"))
)

# Attach surrogate keys
df_fact = (df_fact
    .withColumn("date_sk", sk(["date"]))
    .withColumn("customer_sk", sk(["customer_id"]))
    .withColumn("product_sk", sk(["product_id"]))
    .withColumn("quantity", F.col("quantity").cast("int"))
    .withColumn("unit_price", F.col("unit_price").cast("double"))
    .withColumn("subtotal", F.col("quantity")*F.col("unit_price"))
    .withColumn("year", F.year("date"))
    .withColumn("month", F.month("date"))
    .select("order_id","date_sk","customer_sk","product_sk","quantity","unit_price","subtotal","year","month")
)

df_fact.explain("formatted")
export_spark_metrics_lab2(spark, run_id="r1", task="fact_join", note="build fact_sales with SKs")
with open("proof/plan_fact_join.txt","w") as f:
    from datetime import datetime as _dt
    f.write(str(_dt.now())+"\n")
    f.write(df_fact._jdf.queryExecution().executedPlan().toString())
print("Saved proof/plan_fact_join.txt")


== Physical Plan ==
AdaptiveSparkPlan (19)
+- Project (18)
   +- Project (17)
      +- BroadcastHashJoin Inner BuildRight (16)
         :- Project (12)
         :  +- BroadcastHashJoin Inner BuildRight (11)
         :     :- Project (7)
         :     :  +- BroadcastHashJoin Inner BuildRight (6)
         :     :     :- Filter (2)
         :     :     :  +- Scan csv  (1)
         :     :     +- BroadcastExchange (5)
         :     :        +- Filter (4)
         :     :           +- Scan csv  (3)
         :     +- BroadcastExchange (10)
         :        +- Filter (9)
         :           +- Scan csv  (8)
         +- BroadcastExchange (15)
            +- Filter (14)
               +- Scan csv  (13)


(1) Scan csv 
Output [4]: [order_id#407, product_id#408, quantity#409, unit_price#410]
Batched: false
Location: InMemoryFileIndex [file:/mnt/c/Users/Justine/Data_engineering/lab2/data/lab2_order_items.csv]
PushedFilters: [IsNotNull(product_id), IsNotNull(order_id)]
ReadSchema: struct<order_

## 6. Write Parquet outputs (partitioned by year, month)

In [52]:
base_out = "outputs"
(dim_customer.write.mode("overwrite").parquet(f"{base_out}/dim_customer"))
(dim_brand.write.mode("overwrite").parquet(f"{base_out}/dim_brand"))
(dim_category.write.mode("overwrite").parquet(f"{base_out}/dim_category"))
(dim_product.write.mode("overwrite").parquet(f"{base_out}/dim_product"))
(dim_date.write.mode("overwrite").parquet(f"{base_out}/dim_date"))
(df_fact.write.mode("overwrite").partitionBy("year","month").parquet(f"{base_out}/fact_sales"))
print("Parquet written under outputs")


Parquet written under outputs


## 7. Plan comparison: projection and layout

In [53]:
# Case A: join and then project
a = (orders.join(order_items, "order_id")
            .join(products, "product_id")
            .groupBy(F.to_date("order_date").alias("d"))
            .agg(F.sum(F.col("quantity")*F.col("price")).alias("gmv")))
a.explain("formatted")
_ = a.count()
export_spark_metrics_lab2(spark, run_id="r1", task="caseA_join_then_project", note="join then aggregate")

# Case B: project early
b = (orders.select("order_id","order_date")
            .join(order_items.select("order_id","product_id","quantity"), "order_id")
            .join(products.select("product_id","price"), "product_id")
            .groupBy(F.to_date("order_date").alias("d"))
            .agg(F.sum(F.col("quantity")*F.col("price")).alias("gmv")))
b.explain("formatted")
_ = b.count()
export_spark_metrics_lab2(spark, run_id="r1", task="caseB_project_then_join", note="project early then join")

print("Record Spark UI metrics for both runs in lab2_metrics_log.csv")


== Physical Plan ==
AdaptiveSparkPlan (16)
+- HashAggregate (15)
   +- Exchange (14)
      +- HashAggregate (13)
         +- Project (12)
            +- BroadcastHashJoin Inner BuildRight (11)
               :- Project (7)
               :  +- BroadcastHashJoin Inner BuildLeft (6)
               :     :- BroadcastExchange (3)
               :     :  +- Filter (2)
               :     :     +- Scan csv  (1)
               :     +- Filter (5)
               :        +- Scan csv  (4)
               +- BroadcastExchange (10)
                  +- Filter (9)
                     +- Scan csv  (8)


(1) Scan csv 
Output [2]: [order_id#403, order_date#405]
Batched: false
Location: InMemoryFileIndex [file:/mnt/c/Users/Justine/Data_engineering/lab2/data/lab2_orders.csv]
PushedFilters: [IsNotNull(order_id)]
ReadSchema: struct<order_id:int,order_date:timestamp>

(2) Filter
Input [2]: [order_id#403, order_date#405]
Condition : isnotnull(order_id#403)

(3) BroadcastExchange
Input [2]: [order_id#403, 

## 8. Cleanup

In [54]:
spark.stop()
print("Spark session stopped.")


Spark session stopped.
